In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
from sklearn.inspection import permutation_importance
import seaborn as sns

# Load the Dataset
file_path = "preprocessed_data/outfield_processed.csv"  
df = pd.read_csv(file_path)

X = df.drop(columns=["Player", "player_market_value_euro"], errors="ignore")  
y = df["player_market_value_euro"]

# Split Data into Training and Testing Sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a refined grid search space
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4], 
    'max_features': ['sqrt', 'log2', None]
}

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Perform Grid Search with Cross-Validation (5-fold)
grid_search = GridSearchCV(
    rf_model, param_grid, 
    cv=5, scoring=['r2', 'neg_mean_absolute_error'], 
    refit='r2', n_jobs=-1, verbose=1
)

grid_search.fit(X_train, y_train)

# Extract the best model
best_rf_model = grid_search.best_estimator_

cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results['param_max_depth'] = cv_results['param_max_depth'].astype(str).replace("nan", "None")
cv_results = cv_results.drop_duplicates(subset=['param_n_estimators', 'param_max_depth'])
cv_results = cv_results.groupby(['param_n_estimators', 'param_max_depth'], as_index=False).mean()
cv_results['param_n_estimators'] = cv_results['param_n_estimators'].astype(int)

# Plot R² Score vs. Number of Trees (n_estimators)
plt.figure(figsize=(10, 6))

for depth in cv_results['param_max_depth'].unique():
    subset = cv_results[cv_results['param_max_depth'] == depth]
    plt.plot(subset['param_n_estimators'], subset['mean_test_r2'], marker='o', label=f"max_depth={depth}")

plt.xlabel("Number of Trees (n_estimators)")
plt.ylabel("R² Score")
plt.title("R² Score vs. Number of Trees for Different Tree Depths")
plt.legend(title="Tree Depth")
plt.grid(True)
plt.show()

print("\nBest Hyperparameters Found:")
for param, value in grid_search.best_params_.items():
    print(f"{param}: {value}")

# Show Grid Search Results
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results[['param_n_estimators', 'param_max_depth', 'param_min_samples_split',
                         'param_min_samples_leaf', 'param_max_features', 'mean_test_r2', 'mean_test_neg_mean_absolute_error']]

# Make Predictions
y_pred = best_rf_model.predict(X_test)

# Evaluate Model Performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n📌 Mean Absolute Error (MAE): {mae:.2f}")
print(f"📌 Mean Squared Error (MSE): {mse:.2f}")
print(f"📌 R² Score: {r2:.2f}")

# # Perform Permutation Importance
# perm_importance = permutation_importance(best_rf_model, X_test, y_test, n_repeats=10, random_state=42)

# # Convert results to a DataFrame
# importance_df = pd.DataFrame({
#     'Feature': X.columns,
#     'Importance': perm_importance.importances_mean
# }).sort_values(by='Importance', ascending=False)

# # Plot Permutation Importance
# plt.figure(figsize=(10, 6))
# plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
# plt.xlabel('Permutation Importance Score')
# plt.ylabel('Feature')
# plt.title('Permutation Feature Importance')
# plt.gca().invert_yaxis()  # Highest importance at the top
# plt.show()

# # Feature Importance Analysis from the Model
# feature_importance = best_rf_model.feature_importances_
# sorted_idx = np.argsort(feature_importance)[::-1]

# # Show Top 10 Features
# plt.figure(figsize=(12, 6))
# plt.barh(range(len(sorted_idx[:10])), feature_importance[sorted_idx[:10]], align='center')
# plt.yticks(range(len(sorted_idx[:10])), [X.columns[i] for i in sorted_idx[:10]])
# plt.xlabel("Feature Importance")
# plt.title("Top 10 Most Important Features in Player Valuation")
# plt.gca().invert_yaxis()
# plt.show()

# # Show Bottom 10 Features
# plt.figure(figsize=(12, 6))
# plt.barh(range(len(sorted_idx[-10:])), feature_importance[sorted_idx[-10:]], align='center')
# plt.yticks(range(len(sorted_idx[-10:])), [X.columns[i] for i in sorted_idx[-10:]])
# plt.xlabel("Feature Importance")
# plt.title("Top 10 Least Important Features in Player Valuation")
# plt.gca().invert_yaxis()
# plt.show()

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


KeyboardInterrupt: 